In [ ]:
!pip install pandas

In [29]:
import pandas as pd
import json

In [40]:
def clean_dataframe(df):
    """
    Strips whitespace from column names and string values in the DataFrame.
    Converts common columns to correct types where applicable.
    """
    if df is None:
        return df

    df.columns = df.columns.str.strip()
    for col in df.select_dtypes(include='object').columns:
        df[col] = df[col].str.strip()

    return df

#Question 1:
def calculate_mean_and_std(dataframe, start_year = 2013, end_year = 2018):
    """
    Computes the mean and standard deviation of U.S. population
    between start_year and end_year (inclusive) from the given DataFrame.

    Parameters:
        df (pd.DataFrame): DataFrame containing columns 'Year' and 'Population'
        start_year (int): Start year of the range (inclusive)
        end_year (int): End year of the range (inclusive)

    Returns:
        dict: {'mean': float, 'std_dev': float}
    """
    try:

        dataframe['Year'] = dataframe['Year'].astype(int)
        filtered_df = (dataframe['Year'] >=start_year) & (dataframe['Year']<=end_year)
        filtered_df = dataframe[filtered_df]

        if filtered_df.empty:
            print(f'No data found between years {start_year} and {end_year}.')
            return {'mean': None, 'std_dev': None}

        calculating_mean_population  = filtered_df['Population'].mean()
        calculating_std_population  = filtered_df['Population'].std()

        return {
            'mean': calculating_mean_population,
            'std_dev': calculating_std_population
        }
    except Exception as e:
        print(f'Error computing statistics: {e}')
        return {'mean': None, 'std_dev': None}

#Question 2:
def get_best_year_per_series(timeseries_df):
    """
    For every series_id in the time-series dataframe, find the year with the largest sum of values.
    Returns a dataframe with: series_id, best year, and max summed value.
    """
    # Ensure proper data types
    timeseries_df['value'] = pd.to_numeric(timeseries_df['value'], errors='coerce')
    timeseries_df['year'] = timeseries_df['year'].astype(int)
    timeseries_df['series_id'] = timeseries_df['series_id'].str.strip()

    # Group by series_id and year, sum values
    yearly_sum = timeseries_df.groupby(['series_id', 'year'])['value'].sum().reset_index()

    # For each series_id, get the year with the max value
    idx = yearly_sum.groupby('series_id')['value'].idxmax()
    best_year_df = yearly_sum.loc[idx].reset_index(drop=True)

    return best_year_df.rename(columns={'series_id':'series_id','year': 'best_year', 'value': 'max_value'})


#Question 3:
def get_series_with_population(timeseries_df, population_df, series_id, period = 'Q01'):
    """
    Join time-series data (Part 1) and population data (Part 2) for a specific series_id and period.
    Returns a dataframe with: series_id, year, period, value, and Population.
    """

    # Clean string columns
    timeseries_df['series_id'] = timeseries_df['series_id'].str.strip()
    timeseries_df['period'] = timeseries_df['period'].str.strip()
    population_df['Year'] = population_df['Year'].astype(int)



    # Use query for filtering
    filtered_ts = timeseries_df.query("series_id == @series_id and period == @period").copy()

    print(f'Filtered Timeseries data shape: {filtered_ts.head()}')
    # Rename for join
    population_df = population_df.rename(columns={'Year': 'year'})
    print(f'Population data shape after rename: {population_df.head()}')

    # Merge on 'year'
    result = pd.merge(filtered_ts, population_df[['year', 'Population']], on='year', how='left')
    print(f'Merged data shape: {result.head()}')

    return result[['series_id', 'year', 'period', 'value', 'Population']]



In [43]:
import requests

#Part 1 - S3 File link
bls_data = pd.read_csv('https://rearc-data-quest-v3-blsdata-290950245089.s3.us-east-1.amazonaws.com/part1/pr.data.0.Current',sep='\t')
bls_data.columns = bls_data.columns.str.strip()
print(f'Loaded from S3 with shape {bls_data.shape}')

#Part 2 - S3 File Link
response = 'https://rearc-data-quest-v3-blsdata-290950245089.s3.us-east-1.amazonaws.com/part2/population_data_2025-08-05T18-24-30Z.json'
content = requests.get(response)
data = content.json()

 # Adjust this based on the structure of the JSON file
if isinstance(data, dict) and 'data' in data:
    df = pd.json_normalize(data['data'])
else:
    df = pd.json_normalize(data)
df.columns = df.columns.str.strip()
print(f'Loaded DataFrame with shape {df.shape}')

Loaded from S3 with shape (37002, 5)
Loaded DataFrame with shape (10, 4)


In [44]:
 # 1. Get Mean ans Standard Deviation
stats = calculate_mean_and_std(df)
if stats['mean'] is not None and stats['std_dev'] is not None:
    print(f'Mean: {stats["mean"]:.2f}, Std Dev: {stats["std_dev"]:.2f}')
else:
    print('Failed to compute mean and standard deviation.')

Mean: 322069808.00, Std Dev: 4158441.04


In [45]:
# 2. Get best year for each series_id

best_year_report = get_best_year_per_series(bls_data)
print(f'Best year value per series is: {best_year_report}')

Best year value per series is:        series_id  best_year  max_value
0    PRS30006011       2022     20.500
1    PRS30006012       2022     17.100
2    PRS30006013       1998    705.895
3    PRS30006021       2010     17.700
4    PRS30006022       2010     12.400
..           ...        ...        ...
277  PRS88003192       2002    282.800
278  PRS88003193       2024    860.838
279  PRS88003201       2022     37.200
280  PRS88003202       2022     28.700
281  PRS88003203       2024    583.441

[282 rows x 3 columns]


In [46]:
#3. Get series_id + population report
merged_report = get_series_with_population(bls_data, df, series_id='PRS30006032', period='Q01')
print(f'Best series with population is: {merged_report}')

Timeseries data shape: (37002, 5)
Population data shape: (10, 4)
Filtered Timeseries data shape:         series_id  year period  value footnote_codes
1057  PRS30006032  1995    Q01    0.0            NaN
1062  PRS30006032  1996    Q01   -4.2            NaN
1067  PRS30006032  1997    Q01    2.8            NaN
1072  PRS30006032  1998    Q01    0.9            NaN
1077  PRS30006032  1999    Q01   -4.1            NaN
Population data shape after rename:   Nation ID         Nation  year   Population
0   01000US  United States  2013  316128839.0
1   01000US  United States  2014  318857056.0
2   01000US  United States  2015  321418821.0
3   01000US  United States  2016  323127515.0
4   01000US  United States  2017  325719178.0
Merged data shape:      series_id  year period  value footnote_codes  Population
0  PRS30006032  1995    Q01    0.0            NaN         NaN
1  PRS30006032  1996    Q01   -4.2            NaN         NaN
2  PRS30006032  1997    Q01    2.8            NaN         NaN
3  PRS